In [1]:
import libsbml
import argparse
import os
import io
import tarfile
import glob
import tempfile
import shutil

import sys

import rpSBML
import rpTool
import rpToolServe
import json
import numpy as np
from scipy.optimize import minimize

import rbo

In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [5]:
with open('match_scores.json', 'r') as fp:
    match_scores = json.load(fp)

In [6]:
match_scores

{'measured_64': {'rp_9_2': {'reac_score': 0.05, 'ec_score': 0.75},
  'rp_9_1': {'reac_score': 0.05, 'ec_score': 0.75}},
 'measured_53': {'rp_3_1': {'reac_score': 0.9, 'ec_score': 1.0}},
 'measured_52': {'rp_1_2': {'reac_score': 0.2, 'ec_score': 1.0},
  'rp_1_1': {'reac_score': 0.2, 'ec_score': 1.0}},
 'measured_33': {'rp_2_15': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_24': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_4': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_3': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_8': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_19': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_13': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_21': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_5': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_25': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_12': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_6': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_7': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_14': {'reac_score'

In [7]:
len_meas_steps = {'measured_2': 9, 'measured_4': 9, 'measured_5': 3, 'measured_6': 4, 'measured_7': 4, 'measured_8': 6, 'measured_9': 4, 'measured_10': 8, 'measured_11': 3, 'measured_13': 4, 'measured_14': 6, 'measured_15': 6, 'measured_17': 4, 'measured_18': 6, 'measured_19': 3, 'measured_20': 3, 'measured_21': 9, 'measured_22': 3, 'measured_23': 2, 'measured_24': 4, 'measured_25': 3, 'measured_26': 6, 'measured_27': 4, 'measured_28': 12, 'measured_29': 2, 'measured_30': 2, 'measured_31': 4, 'measured_32': 3, 'measured_33': 2, 'measured_34': 2, 'measured_35': 4, 'measured_36': 4, 'measured_37': 4, 'measured_38': 4, 'measured_39': 4, 'measured_40': 1, 'measured_41': 1, 'measured_42': 2, 'measured_43': 5, 'measured_44': 6, 'measured_45': 3, 'measured_46': 2, 'measured_47': 4, 'measured_48': 1, 'measured_49': 1, 'measured_50': 1, 'measured_51': 1, 'measured_52': 1, 'measured_53': 2, 'measured_54': 6, 'measured_55': 2, 'measured_56': 2, 'measured_57': 5, 'measured_58': 2, 'measured_59': 2, 'measured_60': 5, 'measured_61': 5, 'measured_62': 3, 'measured_63': 3, 'measured_64': 4, 'measured_65': 4, 'measured_66': 6, 'measured_67': 6, 'measured_68': 3, 'measured_69': 5, 'measured_70': 4, 'measured_71': 2, 'measured_72': 3, 'measured_73': 5, 'measured_74': 4, 'measured_75': 3, 'measured_77': 2, 'measured_78': 2, 'measured_79': 5}

In [8]:
len_meas_steps['measured_52']

1

In [9]:
def matchScore(x):
    all_rbo_ext = []
    for meas in match_scores:
        #retreive the file
        res_tar = glob.glob('/home/mdulac/workspace/Galaxy-SynBioCAD/rpOptimise/results/'+meas+'/rpSelenzyme.tar.xz')[0]
        #compile the list of results
        with tempfile.TemporaryDirectory() as tmpOutputFolder:
            top_fileNames = rpToolServe.runGlobalScore_hdd(open(res_tar, 'rb'),
                                                          tmpOutputFolder+'/tmp.tar.xz',
                                                          x[0],
                                                          x[1],
                                                          x[2],
                                                          x[3],
                                                          len_meas_steps[meas],
                                                          10) #this is the topx to return, don't care about it here
            #compile the mean score for the detected pathways
            m_s = {}
            for i in match_scores[meas]:
                m_s[i] = np.mean([match_scores[meas][i]['reac_score'], match_scores[meas][i]['ec_score']])
            rbo_res = rbo.rbo_dict(top_fileNames, m_s, p=0.9)
            all_rbo_ext.append(rbo_res.ext)
            #print('############ '+str(meas)+' ###########')
            #print(top_fileNames)
            #print(m_s)
            #print(rbo_res.ext) #this is the result
    print('############################')
    print(x)
    print(all_rbo_ext)
    print('---> '+str(1.0-np.mean(all_rbo_ext)))
    return 1.0-np.mean(all_rbo_ext)

In [10]:
x0 = np.array([1.0, 1.0, 1.0, 1.0])
res = minimize(matchScore, x0, method='nelder-mead', options={'xatol': 1e-8, 'disp': True})

SystemError: <built-in function readSBMLFromFile> returned a result with an error set

In [13]:
print(top_fileNames)

{'rp_7_1': 0.3105588483158078, 'rp_3_5': 0.32922614534657474, 'rp_7_2': 0.3105588483158078, 'rp_4_3': 0.3293333212748951, 'rp_4_2': 0.3290269412444326, 'rp_4_5': 0.3294632042655237, 'rp_3_2': 0.3287898823254836, 'rp_8_1': 0.32293830166127113, 'rp_6_2': 0.31251708077361035, 'rp_6_1': 0.31251708077361035, 'rp_9_2': 0.3199676766612712, 'rp_5_2': 0.31168458482670514, 'rp_3_1': 0.32909626235594613, 'rp_3_6': 0.3284835022950211, 'rp_5_3': 0.29084568149763323, 'rp_4_1': 0.3293333212748951, 'rp_5_1': 0.31168458482670514, 'rp_3_3': 0.32909626235594613, 'rp_8_2': 0.32293830166127113, 'rp_9_1': 0.3199676766612712, 'rp_4_4': 0.3293333212748951, 'rp_3_4': 0.32909626235594613, 'rp_2_1': 0.4174502923419271, 'rp_4_6': 0.3287205612139701, 'rp_1_1': 0.39704430948333586}


In [25]:
print({'rp_7_1': 0.3105588483158078, 'rp_3_5': 0.32922614534657474, 'rp_7_2': 0.3105588483158078, 'rp_4_3': 0.3293333212748951, 'rp_4_2': 0.3290269412444326, 'rp_4_5': 0.3294632042655237, 'rp_3_2': 0.3287898823254836, 'rp_8_1': 0.32293830166127113, 'rp_6_2': 0.31251708077361035, 'rp_6_1': 0.31251708077361035, 'rp_9_2': 0.1199676766612712, 'rp_5_2': 0.31168458482670514, 'rp_3_1': 0.32909626235594613, 'rp_3_6': 0.3284835022950211, 'rp_5_3': 0.29084568149763323, 'rp_4_1': 0.3293333212748951, 'rp_5_1': 0.31168458482670514, 'rp_3_3': 0.32909626235594613, 'rp_8_2': 0.32293830166127113, 'rp_9_1': 0.9199676766612712, 'rp_4_4': 0.3293333212748951, 'rp_3_4': 0.32909626235594613, 'rp_2_1': 0.4174502923419271, 'rp_4_6': 0.3287205612139701, 'rp_1_1': 0.39704430948333586})

{'rp_7_1': 0.3105588483158078, 'rp_3_5': 0.32922614534657474, 'rp_7_2': 0.3105588483158078, 'rp_4_3': 0.3293333212748951, 'rp_4_2': 0.3290269412444326, 'rp_4_5': 0.3294632042655237, 'rp_3_2': 0.3287898823254836, 'rp_8_1': 0.32293830166127113, 'rp_6_2': 0.31251708077361035, 'rp_6_1': 0.31251708077361035, 'rp_9_2': 0.1199676766612712, 'rp_5_2': 0.31168458482670514, 'rp_3_1': 0.32909626235594613, 'rp_3_6': 0.3284835022950211, 'rp_5_3': 0.29084568149763323, 'rp_4_1': 0.3293333212748951, 'rp_5_1': 0.31168458482670514, 'rp_3_3': 0.32909626235594613, 'rp_8_2': 0.32293830166127113, 'rp_9_1': 0.9199676766612712, 'rp_4_4': 0.3293333212748951, 'rp_3_4': 0.32909626235594613, 'rp_2_1': 0.4174502923419271, 'rp_4_6': 0.3287205612139701, 'rp_1_1': 0.39704430948333586}


In [10]:
ordered_filenames = [k for k, v in sorted(top_fileNames.items(), key=lambda item: item[1])]

In [12]:
print(ordered_filenames)

['rp_5_3', 'rp_7_1', 'rp_7_2', 'rp_5_2', 'rp_5_1', 'rp_6_2', 'rp_6_1', 'rp_9_2', 'rp_9_1', 'rp_8_1', 'rp_8_2', 'rp_3_6', 'rp_4_6', 'rp_3_2', 'rp_4_2', 'rp_3_1', 'rp_3_3', 'rp_3_4', 'rp_3_5', 'rp_4_3', 'rp_4_1', 'rp_4_4', 'rp_4_5', 'rp_1_1', 'rp_2_1']


In [ ]:
rbo_dict(top_fileNames, dict(a=1, b=2, c=2, d=3), p=.9, sort_ascending=True)

In [8]:
from scipy.stats import kendalltau

In [ ]:
kendalltau

In [2]:
rpToolServe.main('/Users/melchior/Downloads/Galaxy913-[rpFBA].tar',
                 'test_out.tar.xz',
                 1.0,
                 1.0,
                 1.0,
                 1.0,
                 5,
                 10,
                 8901.2,
                 -7570.2,
                 999999.0,
                 0.0,
                 'rp_pathway',
                 'obj_biomass',
                 'dfG_prime_m')

AttributeError: module 'rpToolServe' has no attribute 'main'

In [2]:
a = rpToolServe.runGlobalScore_hdd(open('/home/mdulac/Downloads/Galaxy107-[rpSelenzyme].tar', 'rb'),
                                 'test_out.tar.xz',
                                 1.0,
                                 1.0,
                                 1.0,
                                 1.0,
                                 5,
                                 10,
                                 8901.2,
                                 -7570.2,
                                 5.0,
                                 0.0,
                                 'rp_pathway',
                                 'obj_RP1_sink__restricted_biomass',
                                 'dfG_prime_m')

In [4]:
g = {'rule_score': {'reactions': {'RP1': 0.0, 'RP2': 0.0, 'RP3': 0.0}, 'global': 0.0}, 'fba': {'reactions': {'RP1': {'fba_obj_biomass': 0.0, 'fba_obj_RP1_sink__restricted_biomass': 0.0}, 'RP2': {'fba_obj_biomass': 0.0, 'fba_obj_RP1_sink__restricted_biomass': 0.0}, 'RP3': {'fba_obj_biomass': 0.0, 'fba_obj_RP1_sink__restricted_biomass': 0.0}}, 'global': {'fba_obj_biomass': 0.0, 'fba_obj_RP1_sink__restricted_biomass': 0.0}}, 'selenzyme': {'reactions': {'RP1': 0.899, 'RP2': 0.84762, 'RP3': 0.55333}, 'global': 0.7666499999999999}, 'thermo': {'reactions': {'RP1': {'dfG_prime_m': 0.5660434479205154, 'dfG_uncert': 0.5398678866138138, 'dfG_prime_o': 0.5650043839954865}, 'RP2': {'dfG_prime_m': 0.5404499015854908, 'dfG_uncert': 0.5396242391768992, 'dfG_prime_o': 0.5425280294355486}, 'RP3': {'dfG_prime_m': 0.5730892789412517, 'dfG_uncert': 0.5400607446281782, 'dfG_prime_o': 0.5720502150162228}}, 'global': {'dfG_prime_m': 0.5598608761490861, 'dfG_uncert': 0.5398509568062971, 'dfG_prime_o': 0.559860876149086}}}


In [4]:
g.keys()

dict_keys(['rule_score', 'fba', 'selenzyme', 'thermo'])

In [5]:
g['fba']['reactions'].keys()

dict_keys(['RP1', 'RP2', 'RP3'])

In [6]:
g['fba']['global'].keys()

dict_keys(['fba_obj_biomass', 'fba_obj_RP1_sink__restricted_biomass'])

In [11]:
g['fba']['global']

{'fba_obj_biomass': 0.0, 'fba_obj_RP1_sink__restricted_biomass': 0.0}

In [8]:
g['thermo']['reactions'].keys()

dict_keys(['RP1', 'RP2', 'RP3'])

In [9]:
g['thermo']['reactions']

{'RP1': {'dfG_prime_m': 0.5660434479205154,
  'dfG_uncert': 0.5398678866138138,
  'dfG_prime_o': 0.5650043839954865},
 'RP2': {'dfG_prime_m': 0.5404499015854908,
  'dfG_uncert': 0.5396242391768992,
  'dfG_prime_o': 0.5425280294355486},
 'RP3': {'dfG_prime_m': 0.5730892789412517,
  'dfG_uncert': 0.5400607446281782,
  'dfG_prime_o': 0.5720502150162228}}

In [10]:
g['thermo']['global']

{'dfG_prime_m': 0.5598608761490861,
 'dfG_uncert': 0.5398509568062971,
 'dfG_prime_o': 0.559860876149086}

In [ ]:
for reac in g['thermo']['reactions']:
    g['thermo']['global'][reac] += 

In [5]:
g['selenzyme']

{'reactions': {'RP1': 0.899, 'RP2': 0.84762, 'RP3': 0.55333},
 'global': 0.7666499999999999}

In [3]:
j = {'dfG_prime_m': {}, 'dfG_uncert': {}, 'dfG_prime_o': {}, 'path_id': None, 'step_id': None, 'sub_step_id': None, 'rule_score': None, 'smiles': None, 'inchi': None, 'inchikey': None, 'selenzyme': None, 'rule_id': None, 'rule_ori_reac': None, 'global_score': None, 'flux_value': {'units': 'mmol_per_gDW_per_hr', 'value': 0.7637845695834864}}


In [4]:
j

{'dfG_prime_m': {},
 'dfG_uncert': {},
 'dfG_prime_o': {},
 'path_id': None,
 'step_id': None,
 'sub_step_id': None,
 'rule_score': None,
 'smiles': None,
 'inchi': None,
 'inchikey': None,
 'selenzyme': None,
 'rule_id': None,
 'rule_ori_reac': None,
 'global_score': None,
 'flux_value': {'units': 'mmol_per_gDW_per_hr', 'value': 0.7637845695834864}}

In [3]:
h = {'dfG_prime_m': {}, 'dfG_uncert': {}, 'dfG_prime_o': {}, 'path_id': None, 'step_id': None, 'sub_step_id': None, 'rule_score': None, 'smiles': None, 'inchi': None, 'inchikey': None, 'selenzyme': None, 'rule_id': None, 'rule_ori_reac': None, 'global_score': None, 'flux_value': {'units': 'mmol_per_gDW_per_hr', 'value': 0.7637845695834787}}

In [6]:
h['flux_value']['value']

0.7637845695834787

In [4]:
g = {'rule_score': {'reactions': {'RP1': 0.0, 'RP2': 0.0, 'RP3': 0.0}, 'global': 0.0}, 'fba': {'reactions': {'RP1': {'fba_obj_biomass': 0.0, 'fba_obj_RP1_sink__restricted_biomass': 1.411643237152398e-06}, 'RP2': {'fba_obj_biomass': 0.0, 'fba_obj_RP1_sink__restricted_biomass': 1.411643237152398e-06}, 'RP3': {'fba_obj_biomass': 0.0, 'fba_obj_RP1_sink__restricted_biomass': 1.411643237152398e-06}}, 'global': {'fba_obj_biomass': 0.0, 'fba_obj_RP1_sink__restricted_biomass': 0.0}}, 'selenzyme': {'reactions': {'RP1': 0.899, 'RP2': 0.0, 'RP3': 0.81793}, 'global': 1.71693}, 'thermo': {'reactions': {'RP1': {'dfG_prime_m': 0.5660434479205154, 'dfG_uncert': 0.5398678866138138, 'dfG_prime_o': 0.5650043839954865}, 'RP2': {'dfG_prime_m': 0.5866044559365738, 'dfG_uncert': 0.5395284798361183, 'dfG_prime_o': 0.5866044559365738}, 'RP3': {'dfG_prime_m': 0.5126225003815702, 'dfG_uncert': 0.5399679698578291, 'dfG_prime_o': 0.5136615643065991}}, 'global': {'dfG_prime_m': 0.5126225003815702, 'dfG_uncert': 0.0, 'dfG_prime_o': 0.0}}}

In [7]:
g['thermo']['global']

{'dfG_prime_m': 0.5126225003815702, 'dfG_uncert': 0.0, 'dfG_prime_o': 0.0}

In [1]:
import rpSBML

In [2]:
a = '/Users/melchior/Downloads/Galaxy1015-[rpGlobalScore]/rp_1_2.rpsbml.xml'

In [3]:
rpsbml = rpSBML.rpSBML('test')
rpsbml.readSBML(a)

In [4]:
rpsbml

In [5]:
groups = rpsbml.model.getPlugin('groups')
fbc = rpsbml.model.getPlugin('fbc')
rp_pathway = groups.getGroup('rp_pathway')

In [6]:
rpsbml.readRPpathwayIDs()

['RP1', 'RP2']

In [7]:
members = rp_pathway.getListOfMembers()

In [9]:
members[0]

<Member>

In [13]:
objs = fbc.getListOfObjectives()

In [14]:
objs

<ListOfObjectives[3]>

In [15]:
brsynth_dict = rpsbml.readBRSYNTHAnnotation(objs[0].getAnnotation())

In [16]:
brsynth_dict

{'dfG_prime_m': {},
 'dfG_uncert': {},
 'dfG_prime_o': {},
 'path_id': None,
 'step_id': None,
 'sub_step_id': None,
 'rule_score': None,
 'smiles': None,
 'inchi': None,
 'inchikey': None,
 'selenzyme': None,
 'rule_id': None,
 'rule_ori_reac': None,
 'global_score': None,
 'flux_value': {'units': 'mmol_per_gDW_per_hr', 'value': 0.7637845695834804}}

In [17]:
obj = objs[0]

In [19]:
obj.getListOfAllElements()

[<ListOfFluxObjectives[1]>, <FluxObjective>]

In [20]:
obj.getListOfFluxObjectives()

<ListOfFluxObjectives[1]>

In [21]:
b = obj.getListOfFluxObjectives()[0]

In [22]:
b

<FluxObjective>

In [23]:
rpsbml.readBRSYNTHAnnotation(b.getAnnotation())

{'dfG_prime_m': {},
 'dfG_uncert': {},
 'dfG_prime_o': {},
 'path_id': None,
 'step_id': None,
 'sub_step_id': None,
 'rule_score': None,
 'smiles': None,
 'inchi': None,
 'inchikey': None,
 'selenzyme': None,
 'rule_id': None,
 'rule_ori_reac': None,
 'global_score': None,
 'flux_value': {'units': 'mmol_per_gDW_per_hr', 'value': 0.7637845695834804}}

In [24]:
obj.getId()

'obj_biomass'